<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#CAPM" data-toc-modified-id="CAPM-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>CAPM</a></span></li><li><span><a href="#Empirical-test-of-CAPM" data-toc-modified-id="Empirical-test-of-CAPM-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Empirical test of CAPM</a></span><ul class="toc-item"><li><span><a href="#In-other-words,-fit" data-toc-modified-id="In-other-words,-fit-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>In other words, fit</a></span></li><li><span><a href="#Estimating-the-SML" data-toc-modified-id="Estimating-the-SML-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Estimating the SML</a></span></li></ul></li></ul></li><li><span><a href="#Homework" data-toc-modified-id="Homework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Homework</a></span></li></ul></div>

### CAPM

- In this exercise we will fit CAPM with real financial data(stock prices) and test whether the predictions of CAPM are true.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
import pandas_datareader.data as web
import statsmodels.api as sm
from statsmodels.formula.api import ols 

### Empirical test of CAPM

- We will conduct a simple test on whether the predictions of CAPM hold.
    - Asset return is a function of how much risk it is exposed to.
    - Thus, asset returns should be increasing in betas.
- It is done by the following steps.
    - Setting up the sample data
    - Estimating betas
    - Estimating the SML

In [2]:
df = pd.read_excel('beta_data.xls',  index_col=0, parse_dates = True, header=3)

In [3]:
df
# 총 16개의 포트폴리오

,Low Beta,2,3,High Beta,Low Beta.1,2.1,3.1,High Beta.1,Low Beta.2,2.2,...,3.5,High Beta.5,Low Beta.6,2.6,3.6,High Beta.6,Low Beta.7,2.7,3.7,High Beta.7
date,,,,,,,,,,,,,,,,,,,,,
196307,-0.233,-0.471,3.787,0.002,0.863,0.151,-0.597,-2.090,0.353,0.408,...,0.6966,0.5497,0.4947,0.4503,0.5452,0.8183,0.5834,0.5002,0.4438,0.3788
196308,1.124,-0.661,2.007,9.214,2.145,2.878,6.162,6.429,4.328,3.907,...,0.4689,0.6331,0.4935,0.4637,0.5328,0.8075,0.5463,0.5124,0.4356,0.3817
196309,-0.729,5.764,-0.323,-2.947,-1.337,0.236,-3.178,-4.137,-1.121,-2.219,...,0.4521,0.5482,0.4785,0.4424,0.5239,0.7232,0.5181,0.5073,0.4236,0.3678
196310,-0.939,1.462,0.544,2.286,-0.881,-1.154,-1.740,3.197,-0.895,-1.630,...,0.4559,0.5551,0.4871,0.4518,0.5503,0.7749,0.5119,0.5564,0.4403,0.3616
196311,-0.187,-0.706,-0.987,-0.994,-1.203,-0.654,-2.242,1.711,-1.314,-0.799,...,0.4660,0.6216,0.4192,0.4600,0.5125,0.6030,0.5436,0.5544,0.4434,0.3579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199808,-17.179,-16.508,-18.697,-23.085,-15.007,-16.385,-21.545,-26.331,-12.578,-16.167,...,2.0344,0.7662,0.6048,1.0952,1.5151,0.5604,0.5786,0.4059,0.2918,0.3000
199809,-1.690,-0.286,1.150,8.348,-0.335,2.835,4.701,9.024,3.717,4.291,...,3.3527,1.0314,0.6974,1.3192,1.1749,0.9185,0.5625,0.3886,0.3556,0.3756
199810,1.790,1.363,-0.061,5.254,-1.174,1.410,2.798,6.922,1.130,4.353,...,3.5288,1.1082,0.6792,1.2727,0.8448,0.8498,0.5799,0.3654,0.3276,0.3308


In [4]:
pfs = np.mat(df.iloc[:,:16]) # 16 portfolios formed on size and past beta
factors = np.mat(df.iloc[:,16]) # market portfolio
BMs = np.mat(df.iloc[:,17:]) # 16 portfolios formed on B/M ratio and past beta

'''
16개 포트폴리오 하나하나 factor에 넣어주는 것이 아니라
하나의 df를 넣어줌.

'''

T, N = pfs.shape

In [5]:
pfs.shape

(426, 16)

In [6]:
factors.shape

(1, 426)

1-stage time-series regression (estimating beta for each portfolio)

In [7]:
X = sm.add_constant(factors.T)

ts_model = sm.OLS(pfs, X).fit()
alphas = ts_model.params[0]
betas = ts_model.params[1:]
print("beta estimates from 1-stage time-series regression:")
print("rows: small to big, columns: low to high")
np.mat(betas).reshape(4,4)
# x축: 베타의 크기 y축: 회사의 크기

beta estimates from 1-stage time-series regression:
rows: small to big, columns: low to high


matrix([[0.77842172, 0.98245255, 1.13454692, 1.33811698],
        [0.74968325, 1.02779653, 1.21785727, 1.44684187],
        [0.71854073, 1.01907206, 1.20557186, 1.50285895],
        [0.71755303, 0.97620692, 1.18288207, 1.41142603]])

In [8]:
betas[0]

array([0.77842172, 0.98245255, 1.13454692, 1.33811698, 0.74968325,
       1.02779653, 1.21785727, 1.44684187, 0.71854073, 1.01907206,
       1.20557186, 1.50285895, 0.71755303, 0.97620692, 1.18288207,
       1.41142603])

In [9]:
ts_model.params

array([[ 0.87209266,  0.90836198,  0.6696825 ,  0.44212114,  0.37604977,
         0.16316823, -0.00647137, -0.46726443,  0.40286956,  0.21725524,
         0.00202422, -0.43260566,  0.2832331 ,  0.11283406, -0.18890599,
        -0.48956494],
       [ 0.77842172,  0.98245255,  1.13454692,  1.33811698,  0.74968325,
         1.02779653,  1.21785727,  1.44684187,  0.71854073,  1.01907206,
         1.20557186,  1.50285895,  0.71755303,  0.97620692,  1.18288207,
         1.41142603]])

#### In other words, fit

$r_{i,t} - r_{f,t} = \alpha_i + \beta_i \left( r_{M,t} - r_{f,t} \right) + u_{i,t}$ 

and get $\beta_i$

베타 값에 따라 각 포트폴리오의 기대수익률이 증가하는지를 확인

In [10]:
# 2-nd step cross-sectional regression (estimating SML)
mean_ret = np.mat([np.mean(pfs[:,i]) for i in range(16)]) # 수정
betas1 = sm.add_constant(betas.T)
cs_model1 = sm.OLS(mean_ret.T, betas1).fit()

gammas = cs_model1.params.T
stds = cs_model1.bse

print("estimates of gamma0 and gamma1:")
print(gammas)
print("standard errors:")
print(stds)
print("t-values:")
print(gammas/stds)

estimates of gamma0 and gamma1:
[ 1.41661596 -0.08718031]
standard errors:
[0.36592632 0.32754168]
t-values:
[ 3.87131478 -0.26616556]


In [11]:
cs_model1.summary()

C:\Users\LOVE.DESKTOP-5CS2DDB\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.066
Method:                 Least Squares   F-statistic:                   0.07084
Date:                Tue, 17 May 2022   Prob (F-statistic):              0.794
Time:                        13:00:51   Log-Likelihood:                -3.9814
No. Observations:                  16   AIC:                             11.96
Df Residuals:                      14   BIC:                             13.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4166      0.366      3.871      0.002       0.632       2.201
x1            -0.0872      0.328     -0.266      0.794      -0.790       0.615
==============================================================================
Omnibus:                        3.719   Durbin-Watson:                   0.492
Prob(Omnibus):                  0.156   Jarque-Bera (JB):                2.854
Skew:                           0.993   Prob(JB):                        0.240
Kurtosis:                       2.418   Cond. No.                         8.76
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- gamma0과 gamma1이 0을 넘는다고 할 수 없음
- t-value가 2 이상 or -2 이하이면 계수값의 정확도가 높다는 것을 의미함. 
- gamma0의 t-value는 3.87. 따라서 gamma0가 0이라는 귀무가설은 기각. 결국 0이 아니므로 CAPM 모형의 전제인 r0=0를 충족하지 못하게 됨
- 또한 gamma1의 t-value는 -0.266. 따라서 gamma0가 0이라는 귀무가설은 기각하지 못한다. r1>0을 충족하지 못하게 됨.
- CAPM doesn't hold
- t 값이 크다는 것은 표준 편차가 작다는 것이다.  (독립-종속 변수간 상관도 높음)
t 값이 작다는 것은 표준 편차가 크다는 것이다.  (관계 낮음)
그러므로 유의미한 결과가 나오려면 t 값은 커야 한다.

#### Estimating the SML
$\overline{r_i - r_f} = \gamma_0 + \gamma_1\beta_i+\epsilon_i$
- If CAPM holds, $\hat{\gamma_0}=0$ and $\hat{\gamma_1}>0$ should hold.
- 자산의 기대수익률을 설명

The estimated model

$\hat{\overline{r_i - r_f}} = 1.41^{***} - 0.087\beta_i$

- Does CAPM hold?
- Given that $\gamma_1$ is negative and insignificant, CAPM doesn't hold.

CAPM이 성립한다고 할 수 없음

## Homework

- let rf = 0
- Estimate the model $\overline{r_i - r_f} = \gamma_0 + \gamma_1\beta_i+ \gamma_2 s_i^2+\epsilon_i$, where $s_i^2 = {1 \over n-1} (r_i - \bar{r})^2$
- What's the meaning of $\gamma_2$? 
- Interprete the result.
- 위험자산의 표본분산을 추가한 모형
- r^2가 비체계적인 위험일 때 gamma2가 어떤 의미를 지니는지 생각해봐야 함

In [12]:
import pandas as pd

# 2-nd step cross-sectional regression (estimating SML)
mean_ret = np.mat([np.mean(pfs[:,i]) for i in range(16)]) # 수정
sample_var = np.array([np.var(pfs[:,i]) for i in range(16)])

#beta = sm.add_constant(beta.T)

x = np.stack((betas[0], sample_var), axis=1)
x = sm.add_constant(x)
cs_model = sm.OLS(mean_ret.T, x).fit()

In [13]:
gammas = cs_model.params.T
stds = cs_model.bse

print("estimates of gamma0, gamma1, gamma2:")
print(gammas)
print("standard errors:")
print(stds)
print("t-values:")
print(gammas/stds)

estimates of gamma0, gamma1, gamma2:
[ 1.95340041 -1.38024251  0.02350196]
standard errors:
[0.21826619 0.28238634 0.00399164]
t-values:
[ 8.94962431 -4.88778073  5.88779355]


CAPM에서 beta가 체계적 위험을 측정한다면, 분산은 비체계적 위험을 측정할 수 있다. 그러므로 위 회귀식이 성립한다면 포트폴리오의 체계적 위험과 비체계적 위험을 모두 측정할 수 있을 것이다.
- gamma0의 t-value: 8.949이므로 귀무가설이 기각. 1.953이라는 gamma0 추정치는 True. 따라서 r0 = 0이라는 전제를 충족하지 못한다.
- gamma1의 t-value: -4.887이므로 귀무가설이 기각. -1.38이라는 gamma1 추정치는 True. 따라서 r1 > 0이라는 전제를 충족하지 못한다. 
- gamma2의 t-value: 5.888이므로 귀무가설이 기각. 0.023이라는 gamma2 추정치는 True. 그러나 이는 0에 가까운 값이므로 significant한 변수가 될 수 없다.
- r2 is insignificant, so the model doesn't hold.
- 체계적 위험과 비체계적 위험을 반영한 모델이더라도 위험자산 시장의 데이터에는 성립되지 않는다는 것을 알 수 있다.

In [14]:
cs_model.summary()

C:\Users\LOVE.DESKTOP-5CS2DDB\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     17.45
Date:                Tue, 17 May 2022   Prob (F-statistic):           0.000208
Time:                        15:00:04   Log-Likelihood:                 6.4128
No. Observations:                  16   AIC:                            -6.826
Df Residuals:                      13   BIC:                            -4.508
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9534      0.218      8.950      0.000       1.482       2.425
x1            -1.3802      0.282     -4.888      0.000      -1.990      -0.770
x2             0.0235      0.004      5.888      0.000       0.015       0.032
==============================================================================
Omnibus:                        0.040   Durbin-Watson:                   1.481
Prob(Omnibus):                  0.980   Jarque-Bera (JB):                0.217
Skew:                          -0.094   Prob(JB):                        0.897
Kurtosis:                       2.461   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""